In [ ]:
# 📘 08_model_deep_learning.ipynb
# Entraînement d'un modèle de deep learning pour la prédiction de prix

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import joblib
from sentence_transformers import SentenceTransformer

In [ ]:
# 🔹 Chargement des données
file_path = "data/fiverr_cleaned_transformed.csv"
df = pd.read_csv(file_path)
print("✔️ Données chargées :", df.shape)

In [ ]:
# 📥 Chargement des données
df = pd.read_csv("data/fiverr_cleaned_transformed.csv")

# 🧠 Chargement du modèle d'embedding
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 🔹 Embedding de la description
embeddings = embedding_model.encode(df["Description"].astype(str).tolist())
embed_df = pd.DataFrame(embeddings, columns=[f"emb_{i}" for i in range(embeddings.shape[1])])

# 🔹 Encodage one-hot du niveau
niveau_encoded = pd.get_dummies(df["Niveau"], prefix="Niveau")

# 🔹 Variables numériques complémentaires
autres_features = df[["Fiabilite"]].reset_index(drop=True)

# 🔹 Construction du jeu de données final
X = pd.concat([embed_df, niveau_encoded, autres_features], axis=1)
y = df["Prix"]

# 🔹 Séparation train / test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔹 Normalisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🔹 Sauvegarde du scaler
os.makedirs("models/deep", exist_ok=True)
scaler_path = "models/deep/scaler.pkl"
joblib.dump(scaler, scaler_path)

print("📦 Scaler sauvegardé :", scaler_path)
print("✅ Données prêtes pour entraînement deep learning :", X_train_scaled.shape)

In [ ]:
# 🔹 Construction du modèle MLP
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)  # sortie : prix (régression)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
# 🔹 Entraînement avec early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
# 🔹 Évaluation
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"\n📊 Évaluation finale - MAE : {mae:.2f}, MSE : {loss:.2f}")

In [ ]:
# 🔹 Sauvegarde du modèle Keras
model_path = "models/deep/deep_model.h5"
model.save(model_path)
print("✅ Modèle sauvegardé :", model_path)